In [6]:
from onsets_and_frames import *

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, CyclicLR

In [3]:
from fastai import *
from fastai.vision import *

In [4]:
SAMPLE_RATE = 16000
HOP_LENGTH = SAMPLE_RATE * 32 // 1000
ONSET_LENGTH = SAMPLE_RATE * 32 // 1000
OFFSET_LENGTH = SAMPLE_RATE * 32 // 1000
HOPS_IN_ONSET = ONSET_LENGTH // HOP_LENGTH
HOPS_IN_OFFSET = OFFSET_LENGTH // HOP_LENGTH
MIN_MIDI = 21
MAX_MIDI = 108

N_MELS = 229
MEL_FMIN = 30
MEL_FMAX = SAMPLE_RATE // 2
WINDOW_LENGTH = 2048
model_complexity = 48
batch_size = 8
sequence_length=327680

DEFAULT_DEVICE = 'cuda'
N_EPOCHS = 100

In [5]:
base = Path('/home/zach/Data/')
maestrop = base/'maestro/maestro-v2.0.0/'
mapsp = base/'MAPS'

In [6]:
years = [str(x).split('/')[-1] for x in maestrop.ls() if str(x).split('/')[-1].startswith('20')]
years = years[:4]

In [ ]:
val_year = random.choice(years)
train_years = list(set(years)-set([val_year]))

train_ds = MAESTRO(path=maestrop, groups=train_years, sequence_length=sequence_length)
valid_ds = MAESTRO(path=maestrop, groups=[val_year], sequence_length=sequence_length)

Loading 3 groups of MAESTRO at /home/zach/Data/maestro/maestro-v2.0.0


In [11]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=True)

In [12]:
model = OnsetsAndFrames(N_MELS, MAX_MIDI-MIN_MIDI+1, model_complexity)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [14]:
scheduler = CyclicLR(optimizer, base_lr=1e-5, max_lr=1e-3)

In [15]:
from fastprogress import master_bar, progress_bar

In [16]:
mb = master_bar(range(N_EPOCHS))
for epoch in mb:
    for batch in progress_bar(train_dl, parent=mb):
        scheduler.step()
        preds, losses = model.run_on_batch(batch)

        batch_loss = sum(losses.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: cannot pin 'torch.cuda.FloatTensor' only dense CPU tensors can be pinned

In [14]:
?cycle

Signature: cycle(iterable)
Docstring: <no docstring>
File:      ~/Documents/github/onsets-and-frames/onsets_and_frames/utils.py
Type:      function


In [19]:
??torch.optim.lr_scheduler

Type:        module
String form: <module 'torch.optim.lr_scheduler' from '/home/zach/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py'>
File:        ~/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py
Source:     
import types
import math
from torch._six import inf
from functools import partial, wraps
import warnings
from bisect import bisect_right

from .optimizer import Optimizer


class _LRScheduler(object):
    def __init__(self, optimizer, last_epoch=-1):
        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer
        if last_epoch == -1:
            for group in optimizer.param_groups:
                group.setdefault('initial_lr', group['lr'])
            last_epoch = 0
        else:
            for i, group in enumerate(optimizer.param_groups):
                if 'initial_lr' not in group:
                    raise 